In [1]:
# Active env should say "nbai"
!conda info | grep 'active env'

     active environment : nbai
    active env location : C:\Users\percy\anaconda3\envs\nbai


In [2]:
import torch
from datetime import datetime
from torch.nn import functional as F
from torch import nn
from src.graph_format import NbaDatabase, NbaPosessionDatabase

In [3]:
database = NbaPosessionDatabase(range(2001, 2020))

print(len(database))

4584520


In [4]:

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

torch.manual_seed(42)
train_size = int(len(database) * 0.9)
val_size = len(database) - train_size
train_set, val_set = torch.utils.data.random_split(database, [train_size, val_size])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=3200, shuffle=True, pin_memory=use_cuda, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=3200, shuffle=True, pin_memory=use_cuda, num_workers=4)

cpu


In [25]:
class pownet(nn.Module):
    def __init__(self):
        super(pownet, self).__init__()
        self.bn = nn.BatchNorm1d(6)
        self.order_1 = nn.Linear(2, 1, bias=True)
        self.order_2 = nn.Linear(2, 1, bias=True)

    def forward(self, x):
        for _ in range(10):
            x[:, 4][x[:, 4] <= 0] += 300 # overtime
        time_left = (x[:, 4:5] / 2880.0)
        x[:, 2][x[:, 2] > 6] = 6
        x[:, 3][x[:, 3] > 6] = 6
        x = self.bn(x)
        score_diff = x[:, 0] - x[:, 1]
        penalty_diff = x[:, 2] - x[:, 3]
        time_feature = x[:, 4:5]
        state_vec = torch.stack((score_diff, penalty_diff), dim=1)
        order_1 = self.order_1(state_vec)
        order_2 = self.order_2(state_vec) / torch.sqrt(time_left)
        out = torch.sigmoid(order_1 + order_2)
        return out


In [26]:
criterion = nn.BCELoss()
net = pownet()
net = net.to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=1e-1, step_size_up=30)

In [27]:
running_loss = 0
for epoch in range(60):
    epoch_start = datetime.now()
    for start_state, _, label in train_loader:
        start_prob = net(start_state.to(device))
        #end_prob = net(end_state.to(device))
        loss = criterion(start_prob, label.to(device))# + criterion(end_prob, label.to(device))
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()
    seconds = int((datetime.now() - epoch_start).total_seconds())
    print(f"e{epoch}:\tloss {running_loss:.3f}\ttime {seconds}\tlabel {label[0].item()}\tpred {start_prob[0].item():.3f}\tfeat{start_state[0].long().tolist()}")
    running_loss = 0


e0:	loss 652.994	time 405	label 0.0	pred 0.194	feat[16, 29, 4, 2, 2185, 1]
e1:	loss 646.142	time 393	label 0.0	pred 0.010	feat[55, 82, 5, 6, 868, 1]
e2:	loss 646.017	time 384	label 1.0	pred 0.704	feat[82, 78, 2, 5, 871, 1]
e3:	loss 645.806	time 381	label 0.0	pred 0.569	feat[2, 0, 0, 0, 2803, 1]
e4:	loss 645.779	time 384	label 0.0	pred 0.126	feat[69, 78, 1, 2, 570, 1]
e5:	loss 645.627	time 383	label 0.0	pred 0.414	feat[47, 49, 6, 3, 1527, 1]
e6:	loss 645.635	time 385	label 0.0	pred 0.605	feat[14, 12, 0, 5, 2417, 1]
e7:	loss 645.648	time 381	label 0.0	pred 0.553	feat[62, 62, 1, 2, 952, 1]
e8:	loss 645.486	time 382	label 1.0	pred 0.435	feat[49, 52, 3, 6, 1541, 1]
e9:	loss 645.519	time 379	label 0.0	pred 0.041	feat[73, 88, 1, 1, 565, 1]
e10:	loss 645.654	time 382	label 0.0	pred 0.064	feat[51, 67, 1, 2, 965, 1]
e11:	loss 645.607	time 385	label 1.0	pred 0.753	feat[68, 61, 3, 3, 1131, 1]
e12:	loss 645.404	time 379	label 0.0	pred 0.070	feat[74, 88, 0, 0, 702, 1]
e13:	loss 645.378	time 384	labe

In [28]:
# 678 loss
trained_net = net

datetime.timedelta(seconds=5, microseconds=391649)